In [31]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

data_de_hoje = datetime.date.today()

# Coleta os produtos da loja
produtos = []

In [36]:
# Variáveis
service = ChromeService(
    ChromeDriverManager(driver_version="137.0.7151.104").install()
)
driver = webdriver.Chrome(service=service)

driver.get("https://mercado.carrefour.com.br/")

time.sleep(1)

precos_carrefour = []


# Busca pelo nome do producto
search = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Pesquise por produtos ou marcas']"))
)
search.click()
search.send_keys("cerveja")
search.send_keys(Keys.RETURN)

In [37]:
search = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
)
search.click()

# Reire na loja
retire = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
)
retire.click()

# Seleciona a cidade (exemplo: primeira cidade válida)
select = Select(WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
))

total_cidades = len(select.options) - 1  # Subtrai 1 se quiser ignorar a primeira opção ("Selecionar cidade")
print(f"[INFO] Total de cidades disponíveis: {total_cidades}")



[INFO] Total de cidades disponíveis: 55


In [42]:
for i in range(40, 55):
    #### 2 - MANAUS ####
    select.select_by_index(i)  # Exemplo: primeira cidade após o "Selecionar cidade"

    # Aguarda até que as lojas da cidade carreguem no DOM
    loja_elements = WebDriverWait(driver, 2).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4"))
    )
    # Conta e informa o total de lojas
    total_lojas = len(loja_elements)
    print(f"[INFO] Total de lojas na cidade {i}: {total_lojas}")



    for loja_index in range(len(loja_elements)):
        # Recarrega a lista de lojas antes de cada clique
        lojas_atualizadas = driver.find_elements(By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4")
        time.sleep(1)
        loja = lojas_atualizadas[loja_index]
        nome_loja = loja.text.strip()
        print(f"[Cidade {i}] Loja selecionada: {nome_loja}")
        loja.click()
        time.sleep(3)


        # Scroll para carregar todos os produtos
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

        # Captura os blocos de produto
        divs_produtos = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='search-product-card']")

        print(f"[INFO] Total de blocos de produto encontrados: {len(divs_produtos)}")

        for idx in range(len(divs_produtos)):
            try:
                # Recarrega a lista para evitar StaleElementReferenceException
                divs_produtos_atualizados = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='search-product-card']")
                div = divs_produtos_atualizados[idx]

                texto_produto = div.text.strip()

                if texto_produto:
                    # Formata igual ao outro código: produto + loja + cidade
                    registro = f"{texto_produto}\n{nome_loja}"
                    produtos.append(registro)

            except Exception as e:
                print(f"[WARNING] Erro ao coletar produto {idx+1}: {e}")
                continue

        print(f"[INFO] Total de produtos coletados até agora: {len(produtos)}")



        # Antes de voltar para a próxima loja:
        driver.back()
        time.sleep(3)

        # Agora, reabrir o picker de cidade
        # Faz um clique genérico no body para destravar o picker
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class*='container-default']"))
        ).click()
        time.sleep(1)
        
        # Busca por cidade
        search = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
        )
        search.click()
        
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
        ).click()

        # E aqui novamente... recaptura o Select ANTES de re-selecionar a cidade!
        select = Select(WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
        ))
        #select.select_by_index(cidade_index)

        
        select.select_by_index(i)

        time.sleep(2)


driver.quit()

    

[INFO] Total de lojas na cidade 40: 1
[Cidade 40] Loja selecionada: Hiper Nova Piracicaba
Avenida Ruy Teixeira Mendes
Glebas Califórnia, Piracicaba - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos coletados até agora: 1200
[INFO] Total de lojas na cidade 41: 1
[Cidade 41] Loja selecionada: Bairro Praia Grande
Avenida Castelo Branco
Vila Guilhermina, Praia Grande - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos coletados até agora: 1215
[INFO] Total de lojas na cidade 42: 1
[Cidade 42] Loja selecionada: Hiper Prudente
Avenida Manoel Goulart
Jardim das Rosas, Presidente Prudente - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos coletados até agora: 1230
[INFO] Total de lojas na cidade 43: 3
[Cidade 43] Loja selecionada: Hiper Ribeirão Shopping
Avenida Coronel Fernando Ferreira Leite
Jardim California, Ribeirão Preto - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos co

In [43]:
i

54

In [44]:
produtos

['Cerveja Heineken Garrafa 330ml\nR$ 6,79\n-4%\nR$ 6,49\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Heineken Garrafa 600ml\nR$ 12,69\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Baden Baden Golden Ale Garrafa 600ml\nR$ 14,99\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Baden Baden IPA Lata 350ml\nR$ 6,39\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Amstel Lager Lata 473ml\nR$ 4,99\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Eisenbahn Pilsen Long Neck 355ml\nR$ 5,39\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Blue Moon Garrafa 355ml\nR$ 10,99\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió -

In [45]:
len(produtos)

1875

In [46]:
produtos_limpos = []

for p in produtos:
    # Remover linha se for "Produto indisponível" ou "lencol"
    if 'Produto indisponível' in p or 'lencol' in p.lower():
        continue

    # Remover a palavra "ADICIONAR"
    p = p.replace('ADICIONAR', '').strip()

    # Remover qualquer preço no formato por kg, litro, etc
    p = re.sub(r"R\$ ?\d+,\d{2}/(kg|g|l|ml|100ml|un|litro)", "", p, flags=re.IGNORECASE).strip()

    # Se sobrar algo válido, incluir na lista final
    if p:
        produtos_limpos.append(p)

produtos_limpos

['Cerveja Heineken Garrafa 330ml\nR$ 6,79\n-4%\nR$ 6,49\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Heineken Garrafa 600ml\nR$ 12,69\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Baden Baden Golden Ale Garrafa 600ml\nR$ 14,99\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Baden Baden IPA Lata 350ml\nR$ 6,39\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Amstel Lager Lata 473ml\nR$ 4,99\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Eisenbahn Pilsen Long Neck 355ml\nR$ 5,39\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Blue Moon Garrafa 355ml\nR$ 10,99\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Cerveja Baden Baden Witbier Coentro e Laranja Garrafa 6

In [47]:
len(produtos_limpos)

1870

In [48]:
# Lista original
dados_brutos = produtos_limpos

data_de_hoje = data_de_hoje
dados_estruturados = []

for linha in dados_brutos:
    try:
        partes = linha.split("\n\n")
        produto_bloco = partes[0]
        endereco_bloco = partes[1] if len(partes) > 1 else ""

        # Quebra o bloco do produto em linhas individuais
        linhas = produto_bloco.strip().split("\n")

        # REMOVE TRECHOS tipo "/kg", "/g", "/L" apenas de dentro das linhas
        linhas_limpa = []
        for l in linhas:
            # Remove apenas os fragmentos como "R$ XX,XX/kg", "R$ XX,XX/L", etc
            l_limpo = re.sub(r"R\$ ?\d+,\d{2}/(kg|g|l|ml|100ml|un|litro)", "", l, flags=re.IGNORECASE)
            linhas_limpa.append(l_limpo.strip())

        # Nome do produto
        produto_nome = next((x for x in linhas_limpa if "R$" not in x and "%" not in x and "promoção" not in x.lower()), "").strip()

        # Preço final (último R$ válido depois da limpeza)
        precos_encontrados = [x for x in linhas_limpa if re.search(r"R\$ ?\d+,\d{2}", x)]
        preco_final = precos_encontrados[-1] if precos_encontrados else None
        if preco_final:
            preco_final = re.search(r"R\$ ?(\d+,\d{2})", preco_final).group(1)
            preco_final = float(preco_final.replace(",", "."))

        # Desconto
        desconto_linha = next((x for x in linhas_limpa if "%" in x and "-" in x), None)
        desconto = float(desconto_linha.replace("%", "").replace(",", ".").strip()) if desconto_linha else None

        # Loja e endereço
        endereco_linhas = endereco_bloco.strip().split("\n")
        loja = endereco_linhas[0] if len(endereco_linhas) > 0 else ""
        endereco = " - ".join(endereco_linhas[1:]) if len(endereco_linhas) > 1 else ""

        # Cidade e estado
        cidade_estado_texto = endereco_linhas[-1] if len(endereco_linhas) >= 1 else ""
        if "," in cidade_estado_texto and " - " in cidade_estado_texto:
            cidade_texto = cidade_estado_texto.split(",")[-1].strip()
            cidade, estado = cidade_texto.split(" - ")
        else:
            cidade, estado = "", ""

        # Monta o dicionário final
        dados_estruturados.append({
            "produto": produto_nome,
            "preco": preco_final,
            "desconto": desconto,
            "loja": loja.strip(),
            "endereco": endereco_bloco.strip(),
            "cidade": cidade.strip(),
            "estado": estado.strip(),
            "data": data_de_hoje
        })

    except Exception as e:
        print(f"[ERRO] Linha com problema:\n{linha}\n-> {e}")

# DataFrame final
df_produtos = pd.DataFrame(dados_estruturados)

# Garante que a coluna 'data' exista antes de reorganizar
if 'data' not in df_produtos.columns:
    df_produtos['data'] = data_de_hoje

# Reorganizar com 'data' na frente
colunas = ['data'] + [col for col in df_produtos.columns if col != 'data']
df_produtos = df_produtos[colunas]

#Incluir coluna de regioes
regioes_brasil = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste',
    'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}
df_produtos['regiao'] = df_produtos['estado'].map(regioes_brasil).fillna('Desconhecida')

# Alterar os dados da coluna cidade
df_produtos['cidade'] = df_produtos['cidade']+'/'+df_produtos['estado']

df_produtos.rename(columns={'estado': 'uf'}, inplace=True)

df_produtos

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
0,2025-06-18,Cerveja Heineken Garrafa 330ml,6.49,-4.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
1,2025-06-18,Cerveja Heineken Garrafa 600ml,12.69,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
2,2025-06-18,Cerveja Baden Baden Golden Ale Garrafa 600ml,14.99,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
3,2025-06-18,Cerveja Baden Baden IPA Lata 350ml,6.39,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
4,2025-06-18,Cerveja Amstel Lager Lata 473ml,4.99,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
...,...,...,...,...,...,...,...,...,...
1865,2025-06-18,Cerveja Eisenbahn American IPA Lata 350ml,5.69,NaN,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste
1866,2025-06-18,Cerveja Brahma Duplo Malte Puro Malte 350ml Lata,3.49,-13.0,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste
1867,2025-06-18,Cerveja Heineken Barril 5L,119.90,NaN,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste
1868,2025-06-18,Cerveja Budweiser American Lager Lata 269 ml,2.99,-9.0,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste


In [49]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870 entries, 0 to 1869
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1870 non-null   object 
 1   produto   1870 non-null   object 
 2   preco     1870 non-null   float64
 3   desconto  652 non-null    float64
 4   loja      1870 non-null   object 
 5   endereco  1870 non-null   object 
 6   cidade    1870 non-null   object 
 7   uf        1870 non-null   object 
 8   regiao    1870 non-null   object 
dtypes: float64(2), object(7)
memory usage: 131.6+ KB


In [50]:
df_produtos.to_csv('precos_carrefour_cerveja_20250617_bruto.csv', index=False)

In [105]:
df_produtos = pd.read_csv('precos_carrefour_cerveja_20250617_bruto.csv')

In [106]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870 entries, 0 to 1869
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1870 non-null   object 
 1   produto   1870 non-null   object 
 2   preco     1870 non-null   float64
 3   desconto  652 non-null    float64
 4   loja      1870 non-null   object 
 5   endereco  1870 non-null   object 
 6   cidade    1870 non-null   object 
 7   uf        1870 non-null   object 
 8   regiao    1870 non-null   object 
dtypes: float64(2), object(7)
memory usage: 131.6+ KB


In [109]:
df_produtos.loja.unique()

array(['Hiper Maceió Jatiúca', 'Hiper Manaus Manpar',
       'Hiper Manaus Ponta Negra', 'Hiper Manaus Adrianópolis',
       'Hiper Manaus Shopping', 'Hiper Itabuna', 'Hiper Brasília Sul',
       'Hiper Águas Claras', 'Bairro Asa Norte Ii', 'Bairro Bom Motivo',
       'Hiper Asa Norte', 'Hiper Vila Velha', 'Hiper Goiânia Sul',
       'Hiper Avenida T9', 'Hiper Anápolis', 'Hiper Contagem',
       'Hiper BH Shopping', 'Hiper Pampulha', 'Hiper BH América',
       'Hiper Juiz de Fora', 'Hiper Uberlândia', 'Hiper Campo Grande',
       'Hiper Campina Grande', 'Hiper João Pessoa',
       'Hiper João Pessoa Bancários', 'Hiper Shopping Guararapes',
       'Hiper Shopping Petrolina', 'Hiper Avenida Recife',
       'Hiper Shopping Tacaruna', 'Hiper Recife Torre',
       'Hiper Recife Domingos Ferreira', 'Hiper Pinhais',
       'Hiper Champagnat', 'Hiper Parolim', 'Hiper Portão',
       'Hiper Cabral', 'Hiper Londrina', 'Hiper Campos de Goytacazes',
       'Hiper Duque de Caxias', 'Hiper Duque de 

In [110]:
df_lojas_geo = pd.read_csv('lista_lojas_carrefour.csv', sep=';')
df_lojas_geo['loja'].unique()

array(['Hiper Maceió Jatiúca', 'Hiper Manaus Manpar',
       'Hiper Manaus Ponta Negra', 'Hiper Manaus Adrianópolis',
       'Hiper Manaus Shopping', 'Hiper Itabuna', 'Hiper Brasília Sul',
       'Hiper Águas Claras', 'Bairro Asa Norte Ii', 'Bairro Bom Motivo',
       'Hiper Asa Norte', 'Hiper Vila Velha', 'Hiper Goiânia Sul',
       'Hiper Avenida T9', 'Hiper Anápolis', 'Hiper Contagem',
       'Hiper BH Shopping', 'Hiper Pampulha', 'Hiper BH América',
       'Hiper Juiz de Fora', 'Hiper Uberlândia', 'Hiper Campo Grande',
       'Hiper Campina Grande', 'Hiper João Pessoa',
       'Hiper João Pessoa Bancários', 'Hiper Shopping Guararapes',
       'Hiper Shopping Petrolina', 'Hiper Avenida Recife',
       'Hiper Shopping Tacaruna', 'Hiper Recife Torre',
       'Hiper Recife Domingos Ferreira', 'Hiper Pinhais',
       'Hiper Champagnat', 'Hiper Parolim', 'Hiper Portão',
       'Hiper Cabral', 'Hiper Londrina', 'Hiper Campos de Goytacazes',
       'Hiper Duque de Caxias', 'Hiper Duque de 

In [111]:
# Comparar lojas presentes em df_produtos mas não em df_lojas_geo
lojas_produtos = set(df_produtos.loja.unique())
lojas_geo = set(df_lojas_geo['loja'].unique())

lojas_somente_produtos = lojas_produtos - lojas_geo
lojas_somente_geo = lojas_geo - lojas_produtos

print("Lojas em df_produtos e não em df_lojas_geo:", lojas_somente_produtos)
print("Lojas em df_lojas_geo e não em df_produtos:", lojas_somente_geo)

Lojas em df_produtos e não em df_lojas_geo: {'Hiper Tietê', 'Bairro Caçapava'}
Lojas em df_lojas_geo e não em df_produtos: {'Hiper Vila Maria', 'Bairro Vinhedo'}


In [112]:
# Nomes de lojas duplicadas
df_lojas_geo['loja'].value_counts().loc[lambda x: x > 1]


loja
Hiper Guarulhos       3
Hiper Osasco          3
Hiper Santa Maria     2
Hiper Campo Grande    2
Name: count, dtype: int64

In [113]:
df_lojas_geo = pd.read_csv('lista_lojas_carrefour.csv', sep=';')

# Mesclar as informações de geolocalização
df_geo_merge = pd.merge(df_produtos, df_lojas_geo[['loja', 'lat', 'long']],
                        on='loja', how='left')

df_geo_merge['lat'] = df_geo_merge['lat'].astype(str).str.replace(',', '.').astype(float)
df_geo_merge['long'] = df_geo_merge['long'].astype(str).str.replace(',', '.').astype(float)

# Identificar lojas sem geolocalização
lojas_sem_geo = df_geo_merge[df_geo_merge['lat'].isna()]['loja'].unique()

# Remover linhas com qualquer NaN
df_geo_merge = df_geo_merge[df_geo_merge['lat'].notna()]
# Remover linhas duplicadas (com base em todas as colunas)
df_geo_merge = df_geo_merge.drop_duplicates()

lojas_sem_geo.tolist()

['Bairro Caçapava', 'Hiper Tietê']

In [71]:
# Mesclar as informações de geolocalização
df_geo_merge = pd.merge(df_produtos, df_lojas_geo[['loja', 'lat', 'long']],
                        on='loja', how='left')

In [114]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870 entries, 0 to 1869
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1870 non-null   object 
 1   produto   1870 non-null   object 
 2   preco     1870 non-null   float64
 3   desconto  652 non-null    float64
 4   loja      1870 non-null   object 
 5   endereco  1870 non-null   object 
 6   cidade    1870 non-null   object 
 7   uf        1870 non-null   object 
 8   regiao    1870 non-null   object 
dtypes: float64(2), object(7)
memory usage: 131.6+ KB


In [115]:
df_produtos.loc[df_produtos['loja']== 'Hiper Águas Claras']

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
105,2025-06-18,Cerveja Heineken Garrafa 600ml,12.69,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
106,2025-06-18,Cerveja Heineken Garrafa 330ml,6.49,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
107,2025-06-18,Cerveja Baden Baden Golden Ale Garrafa 600ml,12.29,-18.0,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
108,2025-06-18,Cerveja Baden Baden IPA Lata 350ml,5.19,-31.0,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
109,2025-06-18,Cerveja Eisenbahn Pilsen Lata 269ml,3.69,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
110,2025-06-18,Cerveja Blue Moon Garrafa 355ml,11.39,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
111,2025-06-18,Cerveja Baden Baden Witbier Coentro e Laranja ...,14.99,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
112,2025-06-18,Cerveja Lagunitas IPA Lata 350ml,7.99,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
113,2025-06-18,Cerveja Devassa Puro Malte Lata 269ml,2.79,NaN,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste
114,2025-06-18,Cerveja Brahma Duplo Malte Puro Malte 350ml Lata,3.49,-13.0,Hiper Águas Claras,"Hiper Águas Claras\nRua Copaíba\nÁguas Claras,...",Brasília/DF,DF,Centro-Oeste


In [116]:
# Remover linhas duplicadas (com base em todas as colunas)
df_geo_merge = df_geo_merge.drop_duplicates()

In [117]:
df_geo_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1973 entries, 0 to 2107
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1973 non-null   object 
 1   produto   1973 non-null   object 
 2   preco     1973 non-null   float64
 3   desconto  686 non-null    float64
 4   loja      1973 non-null   object 
 5   endereco  1973 non-null   object 
 6   cidade    1973 non-null   object 
 7   uf        1973 non-null   object 
 8   regiao    1973 non-null   object 
 9   lat       1973 non-null   float64
 10  long      1973 non-null   float64
dtypes: float64(4), object(7)
memory usage: 185.0+ KB


In [118]:
df_geo_merge

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao,lat,long
0,2025-06-18,Cerveja Heineken Garrafa 330ml,6.49,-4.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
1,2025-06-18,Cerveja Heineken Garrafa 600ml,12.69,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
2,2025-06-18,Cerveja Baden Baden Golden Ale Garrafa 600ml,14.99,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
3,2025-06-18,Cerveja Baden Baden IPA Lata 350ml,6.39,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
4,2025-06-18,Cerveja Amstel Lager Lata 473ml,4.99,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
...,...,...,...,...,...,...,...,...,...,...,...
2103,2025-06-18,Cerveja Eisenbahn American IPA Lata 350ml,5.69,NaN,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste,-23.02246,-45.57783
2104,2025-06-18,Cerveja Brahma Duplo Malte Puro Malte 350ml Lata,3.49,-13.0,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste,-23.02246,-45.57783
2105,2025-06-18,Cerveja Heineken Barril 5L,119.90,NaN,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste,-23.02246,-45.57783
2106,2025-06-18,Cerveja Budweiser American Lager Lata 269 ml,2.99,-9.0,Hiper Taubaté Charles,Hiper Taubaté Charles\nAvenida Charles Schnnei...,Taubaté/SP,SP,Sudeste,-23.02246,-45.57783


In [119]:
df_geo_merge.loc[df_geo_merge['cidade']== 'Brasília/DF']

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao,lat,long
90,2025-06-18,Cerveja Heineken Garrafa 600ml,12.69,NaN,Hiper Brasília Sul,"Hiper Brasília Sul\nSCEE / Sul\nGuará, Brasíli...",Brasília/DF,DF,Centro-Oeste,-15.82938,-47.95478
91,2025-06-18,Cerveja Heineken Garrafa 330ml,6.49,NaN,Hiper Brasília Sul,"Hiper Brasília Sul\nSCEE / Sul\nGuará, Brasíli...",Brasília/DF,DF,Centro-Oeste,-15.82938,-47.95478
92,2025-06-18,Cerveja Baden Baden Golden Ale Garrafa 600ml,12.29,-18.0,Hiper Brasília Sul,"Hiper Brasília Sul\nSCEE / Sul\nGuará, Brasíli...",Brasília/DF,DF,Centro-Oeste,-15.82938,-47.95478
93,2025-06-18,Cerveja Baden Baden IPA Lata 350ml,5.19,-31.0,Hiper Brasília Sul,"Hiper Brasília Sul\nSCEE / Sul\nGuará, Brasíli...",Brasília/DF,DF,Centro-Oeste,-15.82938,-47.95478
94,2025-06-18,Cerveja Eisenbahn Pilsen Lata 269ml,3.69,NaN,Hiper Brasília Sul,"Hiper Brasília Sul\nSCEE / Sul\nGuará, Brasíli...",Brasília/DF,DF,Centro-Oeste,-15.82938,-47.95478
...,...,...,...,...,...,...,...,...,...,...,...
159,2025-06-18,Cerveja Brahma Duplo Malte Puro Malte 350ml Lata,3.49,-13.0,Hiper Asa Norte,Hiper Asa Norte\nST Setor Terminal Norte\nAsa ...,Brasília/DF,DF,Centro-Oeste,-15.72217,-47.89789
160,2025-06-18,Cerveja Heineken Barril 5L,99.90,-9.0,Hiper Asa Norte,Hiper Asa Norte\nST Setor Terminal Norte\nAsa ...,Brasília/DF,DF,Centro-Oeste,-15.72217,-47.89789
161,2025-06-18,Cerveja Budweiser American Lager Lata 269 ml,2.99,NaN,Hiper Asa Norte,Hiper Asa Norte\nST Setor Terminal Norte\nAsa ...,Brasília/DF,DF,Centro-Oeste,-15.72217,-47.89789
162,2025-06-18,Cerveja Lager Premium Puro Malte Stella Artois...,6.09,NaN,Hiper Asa Norte,Hiper Asa Norte\nST Setor Terminal Norte\nAsa ...,Brasília/DF,DF,Centro-Oeste,-15.72217,-47.89789


In [120]:
df_geo_merge.to_csv('precos_carrefour_cerveja_20250617.csv', index=False)